<br>

<div align=center><font color=maroon size=6><b>Word embeddings</b></font></div>

<br>

<font size=4><b>References:</b></font>
1. TensorFlow > <a href="https://www.tensorflow.org/resources" style="text-decoration:none;">Resources</a> 
    * `TensorFlow > Resources > Text > Guide > `<a href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/guide/word_embeddings.ipynb" style="text-decoration:none;">Word embeddings</a>
        * Run in <a href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/guide/word_embeddings.ipynb" style="text-decoration:none;">Google Colab</a>

<br>
<br>
<br>

This tutorial contains an introduction to word embeddings. You will train your own word embeddings using a simple Keras model for a sentiment classification task, and then visualize them in the [Embedding Projector](http://projector.tensorflow.org) (shown in the image below). 

<img src="images/embedding.jpg" alt="Screenshot of the embedding projector" width="400"/>

<br>

## Representing text as numbers

Machine learning models take vectors (arrays of numbers) as input. When working with text, the first thing you must do is come up with a strategy to convert strings to numbers (or to "vectorize" the text) before feeding it to the model. In this section, you will look at three strategies for doing so.

<br>

### <font style="color:maroon;font-size:110%;">One-hot encodings</font>

As a first idea, you might "one-hot" encode each word in your vocabulary. Consider the sentence "The cat sat on the mat". The vocabulary (or unique words) in this sentence is (cat, mat, on, sat, the). <font style="color:maroon;font-size:120%;">To represent each word, you will create a zero vector with length equal to the vocabulary, then place a one in the index that corresponds to the word.</font> This approach is shown in the following diagram.

<img src="images/one-hot.png" alt="Diagram of one-hot encodings" width="400" />

To create a vector that contains the encoding of the sentence, you could then concatenate the one-hot vectors for each word.

<font style="color:maroon;font-size:120%;">**Key point**: This approach is inefficient. A one-hot encoded vector is sparse (meaning, most indices are zero).</font> Imagine you have 10,000 words in the vocabulary. To one-hot encode each word, you would create a vector where 99.99% of the elements are zero.

<br>
<br>

### <font style="color:maroon;font-size:110%;">Encode each word with a unique number</font>

A second approach you might try is to encode each word using a unique number. Continuing the example above, you could assign 1 to "cat", 2 to "mat", and so on. You could then encode the sentence "The cat sat on the mat" as a dense vector like [5, 1, 4, 3, 5, 2]. <font style="color:maroon;font-size:120%;">This approach is efficient. Instead of a sparse vector, you now have a dense one (where all elements are full).

There are **two downsides** to this approach, however:</font>

* The **integer-encoding** is arbitrary (it does not capture any relationship between words).

* An integer-encoding can be challenging for a model to interpret. A linear classifier, for example, learns a single weight for each feature. Because there is no relationship between the similarity of any two words and the similarity of their encodings, this feature-weight combination is not meaningful.

<br>
<br>

### <font style="color:maroon;font-size:120%;">**Word embeddings**</font>

<font style="color:maroon;font-size:120%;">Word embeddings give us a way to use an efficient, dense representation in which similar words have a similar encoding. Importantly, you do not have to specify this encoding by hand. </font>

<font style="color:maroon;font-size:120%;">An <font style="color:blue;font-size:120%;">**embedding**</font> is **a dense vector of floating point values** (**the length of the vector is a parameter** you specify).</font> Instead of specifying the values for the embedding manually, they are <font style="color:maroon;font-size:120%;">**trainable parameters**</font> (weights learned by the model during training, in the same way a model learns weights for a dense layer). 

<font style="color:maroon;font-size:120%;">It is common to see word embeddings that are **8-dimensional (for small datasets), up to 1024-dimensions when working with large datasets**. A higher dimensional embedding can **capture fine-grained relationships between words**, but takes more data to learn.</font>

<img src="images/embedding2.png" alt="Diagram of an embedding" width="400"/>

Above is a diagram for a word embedding. Each word is represented as a 4-dimensional vector of floating point values.<br> 
<font style="color:maroon;font-size:120%;">Another way to think of an embedding is as "**lookup table**". After these weights have been learned, you can encode each word by looking up the dense vector it corresponds to in the table.</font>

<br>
<br>
<br>

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import TextVectorization

import os
import io
import re
import shutil
import string

In [2]:
print(tf.__version__)

2.8.0


<br>
<br>

### Download the IMDb Dataset
You will use the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) through the tutorial. You will train a sentiment classifier model on this dataset and in the process learn embeddings from scratch. To read more about loading a dataset from scratch, see the [Loading text tutorial](https://www.tensorflow.org/tutorials/load_data/text).  

Download the dataset using Keras file utility and take a look at the directories.

In [3]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", 
                                  url,
                                  untar=True, 
                                  cache_dir='D:/KeepStudy/0_Coding/0_dataset/',
                                  cache_subdir='')

In [4]:
dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

<br>

Take a look at the `train/` directory. It has `pos` and `neg` folders with movie reviews labelled as positive and negative respectively. You will use reviews from `pos` and `neg` folders to train a binary classification model.

In [5]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

<br>

The `train` directory also has additional folders which should be removed before creating training dataset.

In [6]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

<br>

Next, create a `tf.data.Dataset` using `tf.keras.utils.text_dataset_from_directory`. You can read more about using this utility in this [text classification tutorial](https://www.tensorflow.org/tutorials/keras/text_classification). 

Use the `train` directory to create both train and validation datasets with a split of 20% for validation.

In [9]:
batch_size =1024
seed = 123

train_ds = tf.keras.utils.text_dataset_from_directory('D:/KeepStudy/0_Coding/0_dataset/aclImdb/train', 
                                                      batch_size=batch_size, 
                                                      validation_split=0.2,
                                                      subset='training', 
                                                      seed=seed)

val_ds = tf.keras.utils.text_dataset_from_directory('D:/KeepStudy/0_Coding/0_dataset/aclImdb/train',
                                                    batch_size=batch_size,
                                                    validation_split=0.2,
                                                    subset='validation',
                                                    seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


<br>

Take a look at a few movie reviews and their labels `(1: positive, 0: negative)` from the train dataset.

In [11]:
for text_batch, label_batch in train_ds.take(1):
    print(len(text_batch))
    print(text_batch.shape)
    print()
    
    for i in range(5):    # 取 batch 1 的前5条数据
        print(label_batch[i].numpy(), text_batch.numpy()[i])
        print()

1024
(1024,)

0 b"Wow. Some movies just leave me speechless. This was undeniably one of those movies. When I left the theatre, not a single word came to my mouth. All I had was an incredible urge to slam my head against the theatre wall to help me forget about the last hour and a half. Unfortunately, it didn't work. Honestly, this movie has nothing to recommend. The humor was at the first grade level, at best, the acting was overly silly, and the plot was astronomically far-fetched. I hearby pledge never to see an other movie starring Chris Kattan or any other cast-member of SNL."

1 b'If any show in the last ten years deserves a 10, it is this rare gem. It allows us to escape back to a time when things were simpler and more fun. Filled with heart and laughs, this show keeps you laughing through the three decades of difference. The furniture was ugly, the clothes were colorful, and the even the drugs were tolerable. The hair was feathered, the music was accompanied by roller-skates, an

<br>
<br>

### **Configure the dataset for performance**

<font style="color:maroon;font-size:120%;">These are two important methods you should use when loading data to make sure that I/O does not become blocking.</font>

**`.cache()`** keeps data in memory after it's loaded off disk. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache, which is more efficient to read than many small files.

**`.prefetch()`** overlaps data preprocessing and model execution while training. 

You can learn more about both methods, as well as how to cache data to disk in the [data performance guide](https://www.tensorflow.org/guide/data_performance).

In [12]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

<br>
<br>
<br>

## Using the Embedding layer

Keras makes it easy to use word embeddings. Take a look at the [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer.

<font style="color:maroon;font-size:120%;">The Embedding layer can be understood as a **lookup table** that maps from **integer indices** (which stand for specific words) to **dense vectors** (their embeddings).</font> The dimensionality (or width) of the embedding is a parameter you can experiment with to see what works well for your problem, much in the same way you would experiment with the number of neurons in a Dense layer.


In [13]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [14]:
embedding_layer.variables

[]

<br>

When you create an Embedding layer, the weights for the embedding **are randomly initialized** (just like any other layer). During training, they are gradually adjusted via backpropagation. Once trained, the learned word embeddings will roughly encode similarities between words (as they were learned for the specific problem your model is trained on).

<font style="color:maroon;font-size:120%;">If you pass an integer to an embedding layer, the result replaces each integer with the vector from the embedding table:</font>

In [15]:
result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[ 0.00661446,  0.04048273, -0.02802568, -0.01401793, -0.00462662],
       [-0.04271796,  0.02537243,  0.02321133,  0.04151894, -0.01426809],
       [ 0.02120812, -0.0054765 , -0.02212802,  0.01928738, -0.03981546]],
      dtype=float32)

<br>

<font style="color:maroon;font-size:120%;">For text or sequence problems, the Embedding layer takes a 2D tensor of integers, of shape `(samples, sequence_length)`, where each entry is a sequence of integers. It can embed sequences of variable lengths.</font><br>

You could feed into the embedding layer above batches with shapes `(32, 10)` (batch of 32 sequences of length 10) or `(64, 15)` (batch of 64 sequences of length 15).

<font style="color:maroon;font-size:120%;">The returned tensor has one more axis than the input, **the embedding vectors are aligned along the new last axis**. Pass it a `(2, 3)` input batch and the output is `(2, 3, N)`.</font>


In [16]:
result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))   # (2,3). batch of 2 sequences of length 3
result.shape

TensorShape([2, 3, 5])

<br>

<font style="color:maroon;font-size:120%;">When given a batch of sequences as input, an embedding layer returns a 3D floating point tensor, of shape `(samples, sequence_length, embedding_dimensionality)`.</font>

<font style="color:maroon;font-size:120%;">To convert from this sequence of variable length to a fixed representation there are a variety of standard approaches. **You could use an RNN, Attention, or pooling layer before passing it to a Dense layer**. This tutorial uses pooling because it's the simplest. The [Text Classification with an RNN](https://www.tensorflow.org/text/tutorials/text_classification_rnn) tutorial is a good next step.</font>

<br>
<br>
<br>

## Text preprocessing

Next, define the dataset preprocessing steps required for your sentiment classification model. Initialize a TextVectorization layer with the desired parameters to vectorize movie reviews. You can learn more about using this layer in the [Text Classification](https://www.tensorflow.org/tutorials/keras/text_classification) tutorial.

In [21]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

In [22]:
# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

In [23]:
# Use the text vectorization layer to normalize, split, and map strings to integers. 
# Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(standardize=custom_standardization,
                                    max_tokens=vocab_size,
                                    output_mode='int',
                                    output_sequence_length=sequence_length)

In [24]:
# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

<br>
<br>
<br>

## Create a classification model

Use the [Keras Sequential API](https://www.tensorflow.org/guide/keras/sequential_model) to define the sentiment classification model. **In this case it is a "Continuous bag of words" style model**.

* <font style="color:maroon;font-size:110%;">**The [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) layer transforms strings into vocabulary indices**</font>. You have already initialized `vectorize_layer` as a TextVectorization layer and built its vocabulary by calling `adapt` on `text_ds`. Now vectorize_layer can be used as the first layer of your end-to-end classification model, feeding transformed strings into the Embedding layer.


* <font style="color:maroon;font-size:110%;">**The [`Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer takes the integer-encoded vocabulary and looks up the embedding vector for each word-index**</font>. These vectors are learned as the model trains. **The vectors add a dimension to the output array**. The resulting dimensions are: `(batch, sequence, embedding)`.


* <font style="color:maroon;font-size:110%;">**The [`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) layer returns a fixed-length output vector for each example by averaging over the sequence dimension**</font>. This allows the model to handle input of variable length, in the simplest way possible.


* The fixed-length output vector is piped through a fully-connected ([`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)) layer with 16 hidden units.


* The last layer is densely connected with a single output node. 

<font style="color:maroon;font-size:120%;">**Caution**: This model doesn't use masking, so the zero-padding is used as part of the input and hence the padding length may affect the output.  To fix this, see the [masking and padding guide](https://www.tensorflow.org/guide/keras/masking_and_padding).</font>

In [25]:
embedding_dim = 16

model = Sequential([vectorize_layer,
                    Embedding(vocab_size, embedding_dim, name='embedding'),
                    GlobalAveragePooling1D(),
                    Dense(16, activation='relu'),
                    Dense(1)])

<br>
<br>
<br>

## Compile and train the model

You will use [TensorBoard](https://www.tensorflow.org/tensorboard) to visualize metrics including loss and accuracy. Create a `tf.keras.callbacks.TensorBoard`.

In [26]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

<br>

Compile and train the model using the `Adam` optimizer and `BinaryCrossentropy` loss. 

In [27]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [28]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=15,
          callbacks=[tensorboard_callback])

Epoch 1/15
20/20 [==============================] - 4s 128ms/step - loss: 0.6922 - accuracy: 0.5028 - val_loss: 0.6906 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 2s 76ms/step - loss: 0.6880 - accuracy: 0.5028 - val_loss: 0.6847 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 77ms/step - loss: 0.6803 - accuracy: 0.5028 - val_loss: 0.6752 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 77ms/step - loss: 0.6682 - accuracy: 0.5028 - val_loss: 0.6608 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 78ms/step - loss: 0.6504 - accuracy: 0.5031 - val_loss: 0.6413 - val_accuracy: 0.4902
Epoch 6/15
20/20 [==============================] - 2s 77ms/step - loss: 0.6271 - accuracy: 0.5286 - val_loss: 0.6176 - val_accuracy: 0.5448
Epoch 7/15
20/20 [==============================] - 2s 77ms/step - loss: 0.5989 - accuracy: 0.6026 - val_loss: 0.5901 - val_accuracy: 0.6182
Epoch 8/15
2

<br>

With this approach the model reaches a validation accuracy of around 78% (<font style="color:maroon;font-size:120%;">note that the model is overfitting since training accuracy is higher</font>).

<font style="color:maroon;font-size:120%;">**Note**: Your results may be a bit different, depending on how weights were randomly initialized before training the embedding layer. 

You can look into the model summary to learn more about each layer of the model.</font>

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 160,289
Trainable params: 160,289
Non-trai

<br>

Visualize the model metrics in TensorBoard.

In [30]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs

<br>

## Retrieve the trained word embeddings and save them to disk

<font style="color:maroon;font-size:120%;">Next, retrieve the word embeddings learned during training. The embeddings are weights of the Embedding layer in the model. The weights matrix is of shape `(vocab_size, embedding_dimension)`.</font>

<font style="color:maroon;font-size:120%;">Obtain the weights from the model using `get_layer()` and `get_weights()`. The `get_vocabulary()` function provides the vocabulary to build a metadata file with one token per line. </font>

In [31]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

<br>

<font style="color:maroon;font-size:120%;">Write the weights to disk. To use the [Embedding Projector](http://projector.tensorflow.org), you will upload two **files in tab separated format**: 
* a file of **vectors** (containing the embedding), 


* and a file of **meta data** (containing the words).
</font>

In [33]:
# 需要先创建路径：save/Embedding_projector/
out_v = io.open('save/Embedding_projector/vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('save/Embedding_projector/metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
    if index == 0:
        continue    # skip 0, it's padding.
        
    vec = weights[index]
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")

out_v.close()
out_m.close()

<br>

If you are running this tutorial in [Colaboratory](https://colab.research.google.com), you can use the following snippet to download these files to your local machine (or use the file browser, *View -> Table of contents -> File browser*).


```python
try:
    from google.colab import files
    files.download('vectors.tsv')
    files.download('metadata.tsv')
except Exception:
    pass
```

<br>
<br>
<br>

## Visualize the embeddings

<font style="color:maroon;font-size:120%;">To visualize the embeddings, upload them to the embedding projector.

Open the [Embedding Projector](http://projector.tensorflow.org/) (this can also run in a local TensorBoard instance).<br><br>


* Click on "Load data".


* Upload the two files you created above: `vecs.tsv` and `meta.tsv`.
</font>

The embeddings you have trained will now be displayed. You can search for words to find their closest neighbors. For example, try searching for "beautiful". You may see neighbors like "wonderful". 

**Note**: Experimentally, you may be able to produce more interpretable embeddings by using a simpler model. Try deleting the `Dense(16)` layer, retraining the model, and visualizing the embeddings again.

**Note**: Typically, a much larger dataset is needed to train more interpretable word embeddings. This tutorial uses a small IMDb dataset for the purpose of demonstration.


<br>
<br>
<br>

## Next Steps

This tutorial has shown you how to train and visualize word embeddings from scratch on a small dataset.

* To train word embeddings using Word2Vec algorithm, try the [Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec) tutorial. 

* To learn more about advanced text processing, read the [Transformer model for language understanding](https://www.tensorflow.org/text/tutorials/transformer).

<br>
<br>
<br>

**Copyright 2019 The TensorFlow Authors.**

```python
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
```

<br>
<br>
<br>